<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_MultiTimeFrame_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mplfinance

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

df_mypfs = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myPortfolioStocks.csv')
df_mypps = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')

df_mypf = pd.merge(df_mypfs, df_mypps, on="Symbol")

#myts = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myTransactionsReport.csv')

ar_pf = df_mypf[df_mypf['InPortfolio'] == 'DM']

sv_pf = df_mypf[df_mypf['InPortfolio'] == 'SV']


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ', 'EQUITAS']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

all_prospects = get_stock_ids(df_mypps)
ar_stocks = get_stock_ids(ar_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = ar_pf.append(sv_pf)
df_mypps['InPortfolio'] = None
df_mypps.loc[(df_mypps['Symbol'].isin(sv_stocks)), 'InPortfolio'] = 'SV'
df_mypps.loc[(df_mypps['Symbol'].isin(ar_stocks)), 'InPortfolio'] = 'DM'

df_mypps = df_mypps[df_mypps['TFrame'] == 'W']

In [3]:
import pandas as pd
import mplfinance as mpf
import yfinance as yfin
#import talib
yfin.pdr_override()

def get_mtm_data(symbol, tf):
  if tf == 'D':
    intvl, sma, lma, xlma, prd, start_date, tfc = '1d', 20, 50, 200, '2y', '2023-01-01','Daily'
  elif tf == 'W':
    intvl, sma, lma, xlma, prd, start_date, tfc = '1wk', 5, 10, 30, '6y', '2019-01-01','Weekly'
  else:
    intvl, sma, lma, xlma, prd, start_date, tfc = '1mo', 2, 5, 10, '10y', '2013-01-01','Monthly'

  # Sample data with OHLCV (Open, High, Low, Close, Volume) prices
  # Sample data
  df = yfin.Ticker(symbol+'.NS').history(period=prd, interval=intvl)[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  #df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
  #df = pd.DataFrame(data)
  #df.set_index('Date', inplace=True)

  # Calculate moving averages
  df['MA_short'] = df['Close'].rolling(window=sma).mean()
  df['MA_long'] = df['Close'].rolling(window=lma).mean()
  df['MA_xlong'] = df['Close'].rolling(window=xlma).mean()

  df = df[df.index >= start_date]
  col = tf + '_' + 'Flag'
  df[col] = 'R'
  df.loc[df['MA_short'] > df['MA_long'], col] = 'G'
  df = df[df.index > '31-Jan-2023']
  cols = [col]
  return df[df.index > '31-Jan-2023'][cols]

In [4]:

def get_stock_buy_sell(symbol):
  m_df = get_mtm_data(symbol, 'M')
  w_df = get_mtm_data(symbol, 'W')
  d_df = get_mtm_data(symbol, 'D')
  m_df['month'] = m_df.index.month
  w_df['week'] = w_df.index.week
  d_df['week'] = d_df.index.week
  d_df['month'] = d_df.index.month
  d_df = d_df.reset_index()
  cmb_df = pd.merge(d_df, w_df, on='week', how='outer')
  cmb_df = pd.merge(cmb_df, m_df, on='month', how='outer')
  cols = ['Date','D_Flag','W_Flag','M_Flag']
  cmb_df = cmb_df[cols]
  cmb_df.set_index('Date', inplace=True)
  df = yfin.Ticker(symbol+'.NS').history(period='1y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  df = df.join(cmb_df)
  df['F_Flag'] = 'R'
  df['G_Close'] = df['Close'].min()
  df.loc[(df['D_Flag'] == 'G') & (df['W_Flag'] == 'G') & (df['M_Flag'] == 'G'), 'F_Flag'] = 'G'
  df.loc[(df['F_Flag'] == 'G'), 'G_Close'] = df['Close'].max()
  df['Prev_F_Flag'] = df['F_Flag'].shift(1)
  df_reco = df[df['F_Flag'] != df['Prev_F_Flag']]
  df_reco['Signal'] = df_reco['F_Flag'].apply(lambda x: 'InSync' if x == 'G' else 'OutSync')
  df_reco.drop(['Open','Low','High','Volume'], axis=1, inplace=True)
  return df_reco.tail(1)

def get_weekly_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id)
        df_tmp['Stock'] = stock_id
        df_buy_sell = df_buy_sell.append(df_tmp)
    df_buy_sell.drop(['D_Flag','W_Flag','M_Flag','G_Close','Prev_F_Flag'], axis=1, inplace=True)
    return df_buy_sell.sort_index()

df_weekly_buy_sell = get_weekly_buy_sell()
df_weekly_buy_sell

,Close,F_Flag,Signal,Stock
Date,,,,
2023-01-23 00:00:00+05:30,393.299988,R,OutSync,CAMPUS
2023-02-27 00:00:00+05:30,657.952759,R,OutSync,RAJESHEXPO
2023-03-01 00:00:00+05:30,462.269196,G,InSync,AUROPHARMA
2023-05-02 00:00:00+05:30,776.204956,G,InSync,MCDOWELL-N
2023-05-02 00:00:00+05:30,221.467514,G,InSync,NAM-INDIA
...,...,...,...,...
2024-01-23 00:00:00+05:30,3032.250000,R,OutSync,MCX
2024-01-23 00:00:00+05:30,2254.050049,R,OutSync,SRF
2024-01-23 00:00:00+05:30,8195.700195,R,OutSync,TATAELXSI


In [5]:
cols = ['Symbol', 'InPortfolio', 'Criteria', 'Strategy']
tmp_mypf = df_mypps[cols]
tmp_mypf = tmp_mypf.rename({'Symbol': 'Stock'}, axis=1)
tmp_mypf = pd.DataFrame(tmp_mypf.groupby(['Stock', 'Criteria', 'Strategy'])['InPortfolio'].max()).reset_index()
df_weekly_buy_sell = pd.merge(df_weekly_buy_sell.reset_index(), tmp_mypf, on="Stock").set_index('Date')

In [6]:
df_weekly_buy_sell[(df_weekly_buy_sell['Signal'] == 'OutSync') & (df_weekly_buy_sell['InPortfolio'].notnull())].tail(20)

,Close,F_Flag,Signal,Stock,Criteria,Strategy,InPortfolio
Date,,,,,,,
2023-02-27 00:00:00+05:30,657.952759,R,OutSync,RAJESHEXPO,V40N,NTT,DM
2023-07-10 00:00:00+05:30,835.849976,R,OutSync,SBICARD,AR,NTT,DM
2023-08-30 00:00:00+05:30,1447.869507,R,OutSync,LUXIND,V40N,NTT,DM
2023-09-11 00:00:00+05:30,921.450012,R,OutSync,RELAXO,V40N,NTT,DM
2023-09-18 00:00:00+05:30,630.450012,R,OutSync,STARHEALTH,VR,ATH,DM
2023-10-18 00:00:00+05:30,636.224304,R,OutSync,ARE&M,V200,BTT,DM
2023-10-30 00:00:00+05:30,1459.099976,R,OutSync,AAVAS,VR,NTT,DM
2023-10-30 00:00:00+05:30,1578.949951,R,OutSync,NEOGEN,AR,ATH,DM
2023-12-04 00:00:00+05:30,73.150002,R,OutSync,ASIANTILES,VR,NTT,DM


In [7]:
df_weekly_buy_sell[(df_weekly_buy_sell['Signal'] == 'InSync') & (df_weekly_buy_sell['InPortfolio'].isnull())].tail(20)

,Close,F_Flag,Signal,Stock,Criteria,Strategy,InPortfolio
Date,,,,,,,
2023-08-28 00:00:00+05:30,1783.844727,G,InSync,ANGELONE,V40N,ATH,NaN
2023-09-14 00:00:00+05:30,17.350985,G,InSync,INFIBEAM,AR,NTT,NaN
2023-10-12 00:00:00+05:30,1003.750000,G,InSync,MOTILALOFS,V40N,NTT,NaN
2023-10-16 00:00:00+05:30,1093.099976,G,InSync,CAPLIPOINT,V40N,ATH,NaN
2023-11-01 00:00:00+05:30,1223.400024,G,InSync,RADICO,V40N,ATH,NaN
2023-11-07 00:00:00+05:30,444.250000,G,InSync,5PAISA,V40N,BTT,NaN
2023-11-21 00:00:00+05:30,111434.296875,G,InSync,MRF,AR,ATH,NaN
2023-11-28 00:00:00+05:30,110.449997,G,InSync,J&KBANK,VR,NTT,NaN
2023-11-28 00:00:00+05:30,78.550003,G,InSync,PNB,VR,NTT,NaN


In [8]:
df_weekly_buy_sell[df_weekly_buy_sell['Stock'] == 'CUB']

,Close,F_Flag,Signal,Stock,Criteria,Strategy,InPortfolio
Date,,,,,,,
2024-01-15 00:00:00+05:30,151.899994,R,OutSync,CUB,AR,NTT,NaN
